## Workspace Management demo

This notebook presents examples of how to manage and use team workspaces!

In [1]:
import jwt
import requests
import datetime
import boto3
import urllib3
import time
import re
urllib3.disable_warnings()

In [2]:
realm = "eoepca"
base_domain = "apx.develop.eoepca.org"
keycloak_endpoint = f"https://iam-auth.{base_domain}"
workspace_api_endpoint = f'https://workspace-api.{base_domain}/workspaces'
token_endpoint = f"{keycloak_endpoint}/realms/{realm}/protocol/openid-connect/token"
minio_endpoint = "https://minio.develop.eoepca.org"

### token handling

In [3]:
def iam_token(username, password):
    headers = {
        "Cache-Control": "no-cache",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "scope": "roles",
        "grant_type": "password",
        "username": username,
        "password": password,
        "client_id": "demo",
        "client_secret": "demo"
    }    
    response = requests.post(token_endpoint, headers=headers, data=data)
    if response.ok:
        return response.json()["access_token"]
    else:
        print(response)
        return None

**Important**: We expect `example-user-1`, `example-user-2` and `example-admin` to exist in Keycloak!

In [4]:
#the following example users are expected to exist
token_user1 = iam_token("example-user-1", "changeme")
decoded_token_user1 = jwt.decode(token_user1, options={"verify_signature": False})
print(f"\n{decoded_token_user1}")
token_user2 = iam_token("example-user-2", "changeme")
decoded_token_user2 = jwt.decode(token_user2, options={"verify_signature": False})
print(f"\n{decoded_token_user2}")
token_admin = iam_token("example-admin", "changeme")
decoded_token_admin = jwt.decode(token_admin, options={"verify_signature": False})
print(f"\n{decoded_token_admin}")


{'exp': 1744203723, 'iat': 1744203423, 'jti': 'fa5b5f38-b9c6-4791-918f-93f80d59838a', 'iss': 'https://iam-auth.apx.develop.eoepca.org/realms/eoepca', 'aud': ['eoapi', 'resource_registration', 'ws-example-team-1', 'account', 'ws-example-team-2'], 'sub': '0965f986-de8e-43fa-ba1c-36d520eefe6f', 'typ': 'Bearer', 'azp': 'demo', 'sid': 'ae523e3c-acb4-42e9-bf2e-db898e9322b7', 'acr': '1', 'realm_access': {'roles': ['offline_access', 'default-roles-eoepca', 'uma_authorization']}, 'resource_access': {'eoapi': {'roles': ['stac_editor']}, 'resource_registration': {'roles': ['records_editor']}, 'ws-example-team-1': {'roles': ['ws_access']}, 'account': {'roles': ['manage-account', 'view-consent', 'manage-account-links', 'manage-consent', 'view-profile']}, 'ws-example-team-2': {'roles': ['ws_access']}}, 'scope': 'profile email', 'email_verified': False, 'preferred_username': 'example-user-1'}

{'exp': 1744203723, 'iat': 1744203423, 'jti': '28789864-00f2-4caf-b7c8-3acde6f104b2', 'iss': 'https://iam-a

In [5]:
print(decoded_token_user1["aud"])
print(decoded_token_user1["resource_access"])

['eoapi', 'resource_registration', 'ws-example-team-1', 'account', 'ws-example-team-2']
{'eoapi': {'roles': ['stac_editor']}, 'resource_registration': {'roles': ['records_editor']}, 'ws-example-team-1': {'roles': ['ws_access']}, 'account': {'roles': ['manage-account', 'view-consent', 'manage-account-links', 'manage-consent', 'view-profile']}, 'ws-example-team-2': {'roles': ['ws_access']}}


### review existing workspaces

In [6]:
ws_user1 = [entry for entry in decoded_token_user1["aud"] if entry.startswith('ws-')]
print(f"user1: {ws_user1}")
ws_user2 = [entry for entry in decoded_token_user2["aud"] if entry.startswith('ws-')]
print(f"user2: {ws_user2}")

user1: ['ws-example-team-1', 'ws-example-team-2']
user2: []


### workspace creation

In [7]:
preferred_ws_name = f"example-{datetime.datetime.now().timestamp():.0f}"
preferred_ws_name

'example-1744203425'

In [8]:
owner = "example-user-1"
response = requests.post(
    workspace_api_endpoint,
    headers={
        'Authorization': 'Bearer ' + token_admin
    },
    json={
        "preferred_name": preferred_ws_name,
        "default_owner": owner
    }
)
response.raise_for_status()
actual_workspace_name = response.json()['name']
ws_name = response.json()["name"]
print(f"✅ triggered workspace creation '{ws_name}' for '{owner}'")

✅ triggered workspace creation 'ws-example-1744203425' for 'example-user-1'


In [9]:
def access_ws(ws_name, token):
    headers = {
        'Authorization': 'bearer ' + token
    }
    url = f"{workspace_api_endpoint}/{ws_name}"
    print(f"HTTP GET {url}")
    response = requests.get(url, headers=headers)
    print(response)
    #print(response.text)
    return response

In [10]:
while True:
    response = access_ws(ws_name, iam_token("example-user-1", "changeme"))
    if response.status_code == 200:
        try:
            workspace_data = response.json()
            print(workspace_data.get("status"))
            if workspace_data.get("status") == "ready":
                break
        except ValueError:
            print("not ready yet")

    print("...")
    time.sleep(20)    

HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1744203425
<Response [403]>
...
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1744203425
<Response [200]>
provisioning
...
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1744203425
<Response [200]>
provisioning
...
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1744203425
<Response [200]>
ready


### workspace permissions

In [11]:
response1 = access_ws(ws_name, iam_token("example-user-1", "changeme"))
assert response1.status_code == 200
print(f"✅ example-user-1 is workspace owner (see creation above) and can retrieve workspace details")
workspace_data

HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1744203425
<Response [200]>
✅ example-user-1 is workspace owner (see creation above) and can retrieve workspace details


{'status': 'ready',
 'endpoints': [{'id': 'k8s',
   'url': 'k8s-ws-example-1744203425.develop.eoepca.org'},
  {'id': 'workspace-ui',
   'url': 'ws-example-1744203425.apx.develop.eoepca.org'}],
 'storage': {'credentials': {'bucketname': 'ws-example-1744203425',
   'access': 'ws-example-1744203425',
   'secret': 'ETBZwswNTJGr161eCJlpQtaQcbD5owgtEJDDwBr4mpbuCPBIyPuALDhAzyABAHgD',
   'endpoint': 'https://minio.develop.eoepca.org',
   'region': 'eoepca-demo'}},
 'runtime': {'envs': {'AWS_ACCESS_KEY_ID': 'ws-example-1744203425',
   'AWS_ENDPOINT_URL': 'https://minio.develop.eoepca.org',
   'AWS_REGION': 'eoepca-demo',
   'AWS_SECRET_ACCESS_KEY': 'ETBZwswNTJGr161eCJlpQtaQcbD5owgtEJDDwBr4mpbuCPBIyPuALDhAzyABAHgD',
   'KUBECONFIG': 'apiVersion: v1\nclusters:\n- cluster:\n    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURCVENDQWUyZ0F3SUJBZ0lJUllHQ1B6QWhObG93RFFZSktvWklodmNOQVFFTEJRQXdGVEVUTUJFR0ExVUUKQXhNS2EzVmlaWEp1WlhSbGN6QWVGdzB5TlRBME1Ea3hNalUzTWpKYUZ3MHpOVEEwTURjeE1

In [12]:
response2 = access_ws(ws_name, iam_token("example-user-2", "changeme"))
assert response2.status_code == 403
print(f"✅ example-user-2 has not been granted access to retrieve workspace details (yet)")

HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1744203425
<Response [403]>
✅ example-user-2 has not been granted access to retrieve workspace details (yet)


### workspace vcluster (k8s) access 

In [13]:
kubeconfig = workspace_data["runtime"]["envs"]["KUBECONFIG"]

In [14]:
from kubernetes import client, config
from tempfile import NamedTemporaryFile

clean_kubeconfig = re.sub(
    r'^\s*certificate-authority(-data)?:.*\n?', '', 
    kubeconfig.replace("\\n", "\n"), flags=re.MULTILINE
)

# at the moment necessary
patched_kubeconfig = re.sub(
    r'^(?P<indent>\s*)server: .*\n', 
    lambda m: m.group(0) + f"{m.group('indent')}insecure-skip-tls-verify: true\n", 
    clean_kubeconfig, flags=re.MULTILINE
)

with NamedTemporaryFile(mode='w+', delete=False) as temp_kubeconfig:
    temp_kubeconfig.write(patched_kubeconfig)
    temp_kubeconfig.flush()
    print(f"✅ kubeconfig written, use e.g. via `KUBECONFIG={temp_kubeconfig.name} kubectl get pods -A`")

    config.load_kube_config(config_file=temp_kubeconfig.name)
    api_client = client.ApiClient()

✅ kubeconfig written, use e.g. via `KUBECONFIG=/tmp/tmpjq7iuaxd kubectl get pods -A`


In [15]:
v1 = client.CoreV1Api(api_client)
all_pods = v1.list_pod_for_all_namespaces()
print(f"✅ can access (virtual) k8s cluster, the following pods are running:")
for pod in all_pods.items:
    print(f"{pod.metadata.namespace} / {pod.metadata.name}")

✅ can access (virtual) k8s cluster, the following pods are running:
kube-system / coredns-668c87c5d8-kbstf


### workspace s3 access 

In [16]:
bucket_name = workspace_data["storage"]["credentials"]["bucketname"]
s3_access = workspace_data["storage"]["credentials"]["access"]
s3_secret = workspace_data["storage"]["credentials"]["secret"]
s3_endpoint = workspace_data["storage"]["credentials"]["endpoint"]

In [17]:
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=minio_endpoint)

In [18]:
path = 'application-package/s-expression/s-expression-0_0_2.cwl'

In [19]:
object = s3resource.Object(bucket_name, path)
result = object.put(Body=open('../data/s-expression-cwl.cwl', 'rb'))
res = result.get('ResponseMetadata')
if res.get('HTTPStatusCode') == 200:
    print('✅ Application package uploaded successfully')
else:
    print('Application package not uploaded')

✅ Application package uploaded successfully


In [20]:
bucket = s3resource.Bucket(bucket_name)
for obj in bucket.objects.all():
    print(f"Key: {obj.key}, Size: {obj.size} bytes")

Key: application-package/s-expression/s-expression-0_0_2.cwl, Size: 2302 bytes


### workspace ui with storage layer

In [21]:
print(f"You can now open the Workspace UI at https://{ws_name}.{base_domain} and login with example-user-1")

You can now open the Workspace UI at https://ws-example-1744203425.apx.develop.eoepca.org and login with example-user-1


### *!!! bonus !!!* deploy coder (visual studio code server) demonstrating EOAP (EO workflow using CWL)

- documentation: https://eoap.github.io/quickwin/
- github repo (with k8s manifests!) https://github.com/eoap/quickwin/

In [22]:
url = "https://raw.githubusercontent.com/eoap/quickwin/refs/heads/main/coder-manifests/coder.yaml"
response = requests.get(url)
coder_yaml = response.text

coder_yaml = re.sub(r'export AWS_ACCESS_KEY_ID="[^"]+"', f'export AWS_ACCESS_KEY_ID="{s3_access}"', coder_yaml)
coder_yaml = re.sub(r'export AWS_SECRET_ACCESS_KEY="[^"]+"', f'export AWS_SECRET_ACCESS_KEY="{s3_secret}"', coder_yaml)

coder_yaml = re.sub(
    r'- name: AWS_ACCESS_KEY_ID\s*\n\s*value: "[^"]+"',
    f'- name: AWS_ACCESS_KEY_ID\n            value: "{s3_access}"',
    coder_yaml
)
coder_yaml = re.sub(
    r'- name: AWS_SECRET_ACCESS_KEY\s*\n\s*value: "[^"]+"',
    f'- name: AWS_SECRET_ACCESS_KEY\n            value: "{s3_secret}"',
    coder_yaml
)

coder_yaml = re.sub(r'--endpoint-url=http[s]?://[^\s"]+', f'--endpoint-url={s3_endpoint}', coder_yaml)

coder_yaml = re.sub(r'^\s*aws s3 mb s3://results --endpoint-url=.*$', '', coder_yaml, flags=re.MULTILINE)

coder_yaml = re.sub(
    r'(kind: PersistentVolumeClaim[\s\S]+?spec:\s+accessModes:[\s\S]+?requests:\s+storage: \d+Gi)',
    r'\1\n  storageClassName: managed-nfs-storage',
    coder_yaml
)

with NamedTemporaryFile(mode='w+', delete=False) as temp_coder_manifests:
    temp_coder_manifests.write(coder_yaml)
    temp_coder_manifests.flush()

print(f"✅ coder manifests written to: {temp_coder_manifests.name}")

✅ coder manifests written to: /tmp/tmp1kg8fkqh


In [23]:
!KUBECONFIG={temp_kubeconfig.name} kubectl apply -f {temp_coder_manifests.name}

deployment.apps/code-server-deployment created
persistentvolumeclaim/code-server-pvc created
service/code-server-service created
configmap/init created
configmap/bash-rc created
configmap/bash-login created


In [24]:
raise Exception("stop at this cell for demo")

Exception: stop at this cell for demo

### workspace deletion

In [ ]:
result = object.delete()

In [ ]:
response = requests.delete(
    f"{workspace_api_endpoint}/{ws_name}",
    headers={
        'Authorization': 'Bearer ' + iam_token("example-admin", "changeme")
    }
)
response.raise_for_status()
print(f"deleted workspace '{ws_name}'")